In [16]:
import os
import shelve
import datetime
from subprocess import run, PIPE
import numpy as np
from kapascan.measurement import Measurement

In [18]:
host_controller = '192.168.254.173'
host_logger = '192.168.254.174'
serial_port = '/dev/ttyACM0'

settings = {
    'sensors': ['1739'],
    'sampling_time': 0.256,
    'data_points': 50,
    'extent': ((15.5, 21.5, 2), (8.0, 14, 2)),
    'mode': 'absolute',
    'direction': ('x', 'y'),
    'change_direction': False
    }

base_dir = "data"
measurement_dir = "test"
script_filename = "surface.ipynb"

In [19]:
data_dir = os.path.join(base_dir, measurement_dir)
if not os.path.exists(data_dir):
    os.makedirs(data_dir, exist_ok=True)

for i in range(2):
    m = Measurement(host_controller, serial_port, host_logger, settings)
    with m:
        x, y, z, T = m.scan()

    filename = "{:03d}".format(i)
    for coord, data in zip(("x", "y", "z", "T"), (x, y, z, T)):
        np.save(os.path.join(data_dir, filename + "_" + coord), data)
    with shelve.open(os.path.join(data_dir, filename + "_settings")) as file:
        file['settings'] = m.settings

    commit_message = "Measurement {} in {}.".format(i, measurement_dir)
    response = run(["./git.sh", data_dir, script_filename, commit_message], stdout=PIPE, stderr=PIPE)
    print(response.stdout)
    print(response.stderr)
    if response.returncode != 0:
        raise Exception

FileNotFoundError: [Errno 2] No such file or directory: './git.sh'